In [0]:
import requests
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from time import sleep



BASE = "https://olinda.bcb.gov.br/olinda/servico/Pix_DadosAbertos/versao/v1/odata"

schema_transacoes = StructType([
    StructField("AnoMes", IntegerType(), True),
    StructField("Municipio_Ibge", IntegerType(), True),
    StructField("Municipio", StringType(), True),
    StructField("Estado_Ibge", IntegerType(), True),
    StructField("Estado", StringType(), True),
    StructField("Sigla_Regiao", StringType(), True),
    StructField("Regiao", StringType(), True),
    StructField("VL_PagadorPF", DoubleType(), True),
    StructField("QT_PagadorPF", DoubleType(), True),
    StructField("VL_PagadorPJ", DoubleType(), True),
    StructField("QT_PagadorPJ", DoubleType(), True),
    StructField("VL_RecebedorPF", DoubleType(), True),
    StructField("QT_RecebedorPF", DoubleType(), True),
    StructField("VL_RecebedorPJ", DoubleType(), True),
    StructField("QT_RecebedorPJ", DoubleType(), True),
    StructField("QT_PES_PagadorPF", DoubleType(), True),
    StructField("QT_PES_PagadorPJ", DoubleType(), True),
    StructField("QT_PES_RecebedorPF", DoubleType(), True),
    StructField("QT_PES_RecebedorPJ", DoubleType(), True)
])

def fetch_all_data(url_base, date_param_name, date_param_value):
    """
    Função para buscar todos os dados de uma API OData usando o '@odata.nextLink'.
    """
    all_data = []
    # Adicionando o parâmetro '$select=*' para retornar todas as colunas
    initial_url = f"{url_base}({date_param_name}=@{date_param_name})?@{date_param_name}='{date_param_value}'&$format=json&$select=*"
    url = initial_url
    retries = 0
    max_retries = 5

    while url:
        try:
            print(f"Buscando URL: {url}")
            r = requests.get(url, timeout=30)
            r.raise_for_status()
            response_json = r.json()
            
            data = response_json.get('value', [])
            all_data.extend(data)
            
            
            url = response_json.get('@odata.nextLink')
            
            if url:
                print(f"Buscando próxima página para {date_param_value}...")
                sleep(1) 
            
        except requests.exceptions.RequestException as e:
            if retries < max_retries:
                print(f"Erro na requisição para {date_param_value}: {e}. Tentando novamente em {2**retries} segundos...")
                sleep(2**retries)
                retries += 1
            else:
                print(f"Erro persistente para {date_param_value} após {max_retries} tentativas. Abortando a coleta deste período.")
                return []

    return all_data

# --- Coleta e Processamento de Transações por Município ---
print("Iniciando coleta de Transações por Município...")

for year in range(2023, 2025):
    for month in range(1, 13):
        month_str = f"{year}{month:02d}"
        print(f"\nProcessando dados do mês: {month_str}")

        transacoes_do_mes = fetch_all_data(f"{BASE}/TransacoesPixPorMunicipio", "DataBase", month_str)

        if transacoes_do_mes:
            print(f"Transações por município coletadas para {month_str}: {len(transacoes_do_mes)} registros")
            
            try:
                df_spark_transacs = spark.createDataFrame(transacoes_do_mes, schema=schema_transacoes)
                df_spark_transacs.write.format("delta").mode("append").saveAsTable("estatisticas_pix.raw_data.transacoes_pix_por_municipio")
                print(f"Dados do mês {month_str} salvos com sucesso.")

            except Exception as e:
                print(f"Erro ao criar DataFrame Spark ou salvar na tabela para {month_str}: {e}")

            del transacoes_do_mes
        else:
            print(f"Nenhum registro encontrado para {month_str}.")
        
        sleep(2)

print("\nProcesso ETL concluído.")